<a href="https://colab.research.google.com/github/daredevil007-aks/Forex_prediction/blob/main/Copy_of_model_trainer_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikeras[tensorflow] --quiet
!pip3 install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 623.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 22.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.15.0 requires tensorflow<2.16,>=2.15.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.17.0 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.17.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.5 MB/s eta 0:00

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            12Gi       702Mi       7.5Gi       1.0Mi       4.5Gi        11Gi
Swap:             0B          0B          0B


In [ ]:
import psutil
import os

def print_memory():
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2.**30  # memory use in GB
    print(f'Memory usage: {memoryUse:.2f} GB')

print_memory()


Memory usage: 0.09 GB


In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

# Load data
data = pd.read_csv('DAT_MS_EURUSD_M1_202407.csv')

In [ ]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Convert `Datetime` to datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], unit='ms')

# Sort data by datetime just in case
data = data.sort_values(by='Datetime')

# Use only the 'Close' prices for this example
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

NameError: name 'pd' is not defined

In [ ]:
look_back = 24 * 60  # 5 months of minute data (approx. 30 days per month) -> changing to 24 hours for testing hptuning
predict_forward = 5  # Predict next 5 minutes

# Ensure there are enough data points
if len(scaled_prices) < look_back + predict_forward:
    raise ValueError("Not enough data points for the given look_back period.")

# Create dataset
def create_minute_dataset(dataset, look_back, predict_forward=5):
    X, Y = [], []
    for i in range(len(dataset) - look_back - predict_forward):
        # time.sleep(1)
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[(i + look_back):(i + look_back + predict_forward), 0])
    return np.array(X), np.array(Y)

X, Y = create_minute_dataset(scaled_prices, look_back, predict_forward)

In [ ]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Model creation function with **kwargs to accept dynamic parameters
def create_model(units=50, optimizer='adam', **kwargs):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(look_back, 1)))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(predict_forward))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Create the KerasRegressor with model creation function
model = KerasRegressor(model=create_model, verbose=0)

#Define the grid search parameters
param_grid = {
    'model__units': [50, 100],         # Use 'model__' prefix for parameters of the model function
    'model__optimizer': ['adam', 'rmsprop'],
    'epochs': [10, 20],
    'batch_size': [16, 32]
}

# Create GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Fit the model
X_small = X[:1000]
Y_small = Y[:1000]
# grid_result = grid.fit(X_small, Y_small)


# Output the best parameters
# print(f"Best parameters: {grid_result.best_params_}")


In [ ]:
import tensorflow as tf

# # Extract the best hyperparameters
# best_params = grid_result.best_params_

# # Retrain the model using the best parameters
# best_model = grid_result.best_estimator_

# Ensure GPU is used (if available) when fitting the model
with tf.device('/GPU:0'):  # This ensures the model runs on the first available GPU
    # Fit the model on the full dataset
    model.fit(X_small, Y_small, epochs=20, batch_size=16)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Save the trained model
model.model_.save('best_lstm_model.h5')


In [ ]:
from google.colab import files

files.download('best_lstm_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
saved_model = load_model('best_lstm_model.h5')

# Use the loaded model to make predictions
predictions = saved_model.predict(X[-1000:])  # X_new is the new data for which you want predictions


In [ ]:
import matplotlib.pyplot as plt

# Reshape the predictions and actual values for inverse scaling
predictions = np.reshape(predictions, (predictions.shape[0], predict_forward))

# Inverse scale the predicted values
predicted_prices = scaler.inverse_transform(predictions)

# Inverse scale the actual values for comparison
actual_prices = scaler.inverse_transform(Y[-1000:])

# Plot the actual vs predicted prices
plt.figure(figsize=(14, 5))
plt.plot(actual_prices.flatten(), label="Actual Prices", color="blue")
plt.plot(predicted_prices.flatten(), label="Predicted Prices", color="red")
plt.title("Forex Price Prediction (Actual vs Predicted)")
plt.xlabel("Time (minutes)")
plt.ylabel("Price")
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset  # Hugging Face datasets library

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Enable mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Load dataset using Hugging Face's datasets library
dataset = load_dataset('csv', data_files='DAT_MS_EURUSD_M1_202407.csv')
data = pd.DataFrame(dataset['train'])


# Convert `Datetime` to datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%Y%m%d%H%M')

# Use only the 'Close' prices for this example
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Check the length of the scaled data
print(f"Length of minute-level data: {len(scaled_prices)}")

# Set up the look-back period (using minute-level data)
look_back = 2 * 30 * 24 * 60  # 2 months (2 * 30 days * 24 hours * 60 minutes)
predict_forward = 5  # Predict next 5 minutes

# Ensure there are enough data points
if len(scaled_prices) < look_back + predict_forward:
    raise ValueError(f"Not enough data points! Got {len(scaled_prices)}, but need at least {look_back + predict_forward}")

# Create the dataset generator
def data_generator(dataset, look_back, predict_forward=5, batch_size=32):
    data_len = len(dataset)

    while True:
        X, Y = [], []
        for _ in range(batch_size):
            i = np.random.randint(0, data_len - look_back - predict_forward)
            X.append(dataset[i:i + look_back, 0])
            Y.append(dataset[i + look_back:i + look_back + predict_forward, 0])

        X = np.array(X).reshape(batch_size, look_back, 1)
        Y = np.array(Y)
        yield X, Y

# Create the data generator
train_generator = data_generator(scaled_prices, look_back, predict_forward, batch_size=32)
steps_per_epoch = len(scaled_prices) // 32

# Simplified model creation function
def create_model(units=50, optimizer='adam', **kwargs):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(look_back, 1)))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(predict_forward))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Keras Regressor with GridSearchCV
model = KerasRegressor(model=create_model, verbose=0)

# Grid search parameters
param_grid = {
    'model__units': [50, 100],
    'model__optimizer': ['adam', 'rmsprop'],
    'epochs': [10, 20],
    'batch_size': [16, 32]
}


# GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Fitting a smaller subset for hyperparameter tuning
X_small = np.array([scaled_prices[i:i + look_back] for i in range(1000)])
Y_small = np.array([scaled_prices[i + look_back:i + look_back + predict_forward] for i in range(1000)])
X_small = np.reshape(X_small, (X_small.shape[0], X_small.shape[1], 1))

# Fit GridSearchCV on a small subset
grid_result = grid.fit(X_small, Y_small)

# Best parameters
print(f"Best parameters: {grid_result.best_params_}")

# Early stopping and model checkpointing
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
checkpoint = ModelCheckpoint('best_lstm_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Use TPU if available for training
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

# Train the model using the best parameters within TPU strategy
with strategy.scope():
    best_model = grid_result.best_estimator_
    best_model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        callbacks=[early_stop, checkpoint]
    )

# Save the best model
best_model.model_.save('best_lstm_model.h5')



Length of minute-level data: 191413


ValueError: 
All the 48 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 925, in _fit
    X, y = self._initialize(X, y)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 853, in _initialize
    X, y = self._validate_data(X, y, reset=True)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 1733, in _validate_data
    retur

working with gridsearch

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset  # Hugging Face datasets library

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import mixed_precision

# Enable mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Load dataset using Hugging Face's datasets library
dataset = load_dataset('csv', data_files='DAT_MS_EURUSD_M1_202407.csv')
data = pd.DataFrame(dataset['train'])

# Convert `Datetime` to datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%Y%m%d%H%M')

# Use only the 'Close' prices for this example
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Check the length of the scaled data
print(f"Length of minute-level data: {len(scaled_prices)}")

# Set up the look-back period (using minute-level data)
look_back = 2 * 30 * 24 * 60  # 2 months (2 * 30 days * 24 hours * 60 minutes)
predict_forward = 5  # Predict next 5 minutes

# Ensure there are enough data points
if len(scaled_prices) < look_back + predict_forward:
    raise ValueError(f"Not enough data points! Got {len(scaled_prices)}, but need at least {look_back + predict_forward}")

# Create the dataset generator
def data_generator(dataset, look_back, predict_forward=5, batch_size=32):
    data_len = len(dataset)

    while True:
        X, Y = [], []
        for _ in range(batch_size):
            i = np.random.randint(0, data_len - look_back - predict_forward)
            X.append(dataset[i:i + look_back, 0])
            Y.append(dataset[i + look_back:i + look_back + predict_forward, 0])

        X = np.array(X).reshape(batch_size, look_back, 1)
        Y = np.array(Y)
        yield X, Y

# Create the data generator
train_generator = data_generator(scaled_prices, look_back, predict_forward, batch_size=32)
steps_per_epoch = len(scaled_prices) // 32

# Custom wrapper for reshaping data for LSTM models in GridSearchCV
from tensorflow.keras.layers import LSTM

# Modify the create_model function to avoid using CuDNN in LSTM layers
class LSTMRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, units=50, optimizer='adam', epochs=10, batch_size=32):
        self.units = units
        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.model_ = None  # This will store the trained model

    def create_model(self):
        model = Sequential()
        # Disable CuDNN usage by setting use_cudnn=False
        model.add(LSTM(self.units, return_sequences=True, input_shape=(look_back, 1), use_cudnn=False))
        model.add(LSTM(self.units, return_sequences=False, use_cudnn=False))
        model.add(Dense(25))
        model.add(Dense(predict_forward, dtype='float32'))  # Mixed precision output as float32
        model.compile(optimizer=self.optimizer, loss='mean_squared_error')
        return model

    def fit(self, X, y):
        # Reshape the input back to 3D for LSTM
        X = X.reshape((X.shape[0], look_back, 1))
        self.model_ = self.create_model()
        self.model_.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self

    def predict(self, X):
        # Reshape the input back to 3D for prediction
        X = X.reshape((X.shape[0], look_back, 1))
        return self.model_.predict(X)


# Keras Regressor with GridSearchCV
model = LSTMRegressor()

# Grid search parameters
param_grid = {
    'units': [50],
    'optimizer': ['adam'],
    'epochs': [10],
    'batch_size': [8, 16]  # Reduce batch size
}

# GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Fitting a smaller subset for hyperparameter tuning
X_small = np.array([scaled_prices[i:i + look_back] for i in range(1000)])
Y_small = np.array([scaled_prices[i + look_back:i + look_back + predict_forward] for i in range(1000)])
X_small = np.reshape(X_small, (X_small.shape[0], X_small.shape[1], 1))

# Fit GridSearchCV
grid_result = grid.fit(X_small, Y_small)

# Best parameters
print(f"Best parameters: {grid_result.best_params_}")

# Early stopping and model checkpointing
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
checkpoint = ModelCheckpoint('best_lstm_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Use TPU if available for training
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
except Exception as e:
    print("TPU not found, using default strategy:", e)
    strategy = tf.distribute.get_strategy()  # Use default strategy (e.g., GPU or CPU)

# Train the model using the best parameters within TPU strategy
with strategy.scope():
    best_model = grid_result.best_estimator_
    best_model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        callbacks=[early_stop, checkpoint]
    )

# Save the best model
best_model.model_.save('best_lstm_model.h5')


Length of minute-level data: 191413


KeyboardInterrupt: 

without grid serach


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset  # Hugging Face datasets library
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import mixed_precision

# Enable mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Load dataset using Hugging Face's datasets library
dataset = load_dataset('csv', data_files='DAT_MS_EURUSD_M1_202407.csv')
data = pd.DataFrame(dataset['train'])

# Convert `Datetime` to datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%Y%m%d%H%M')

# Use only the 'Close' prices for this example
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Check the length of the scaled data
print(f"Length of minute-level data: {len(scaled_prices)}")

# Set up the look-back period (using minute-level data)
look_back = 2 * 30 * 24 * 60  # 2 months (2 * 30 days * 24 hours * 60 minutes)
predict_forward = 5  # Predict next 5 minutes

# Ensure there are enough data points
if len(scaled_prices) < look_back + predict_forward:
    raise ValueError(f"Not enough data points! Got {len(scaled_prices)}, but need at least {look_back + predict_forward}")

# Create the dataset generator
def data_generator(dataset, look_back, predict_forward=5, batch_size=32):
    data_len = len(dataset)

    while True:
        X, Y = [], []
        for _ in range(batch_size):
            i = np.random.randint(0, data_len - look_back - predict_forward)
            X.append(dataset[i:i + look_back, 0])
            Y.append(dataset[i + look_back:i + look_back + predict_forward, 0])

        X = np.array(X).reshape(batch_size, look_back, 1)
        Y = np.array(Y)
        yield X, Y

# Create the data generator
train_generator = data_generator(scaled_prices, look_back, predict_forward, batch_size=32)
steps_per_epoch = len(scaled_prices) // 32

# LSTM model definition
def create_model(units=50, optimizer='adam'):
    model = Sequential()
    model.add(Input(shape=(look_back, 1)))  # Use Input layer instead of specifying input_shape in LSTM
    model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(predict_forward, dtype='float32'))  # Mixed precision output as float32
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Use TPU if available for training
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU connected and initialized.")
except Exception as e:
    print("Error: TPU not available or not connected properly.", e)
    strategy = tf.distribute.get_strategy()  # Use default strategy (e.g., GPU or CPU)

# Early stopping and model checkpointing
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model using TPU strategy
with strategy.scope():
    model = create_model(units=50, optimizer='adam')  # Adjust units and optimizer if needed
    model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        callbacks=[early_stop, checkpoint]
    )

# Save the best model
model.save('best_lstm_model.keras')


Generating train split: 0 examples [00:00, ? examples/s]

Length of minute-level data: 191413
Error: TPU not available or not connected properly. Please provide a TPU Name to connect to.
Epoch 1/20


# Checking TF TPU

In [ ]:
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("TPU connected successfully!")
except Exception as e:
    print("Error: TPU not available.", e)


Error: TPU not available. Please provide a TPU Name to connect to.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset  # Hugging Face datasets library
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import mixed_precision

# Enable mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Load dataset using Hugging Face's datasets library
dataset = load_dataset('csv', data_files='DAT_MS_EURUSD_M1_202407.csv')
data = pd.DataFrame(dataset['train'])

# Convert `Datetime` to datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%Y%m%d%H%M')

# Use only the 'Close' prices for this example
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Check the length of the scaled data
print(f"Length of minute-level data: {len(scaled_prices)}")

# Set up the look-back period
look_back = 3 * 30 * 24 * 60  # 2 months (adjust this to reduce)
predict_forward = 5  # Predict next 5 minutes

# Ensure there are enough data points
if len(scaled_prices) < look_back + predict_forward:
    raise ValueError(f"Not enough data points! Got {len(scaled_prices)}, but need at least {look_back + predict_forward}")

# Create the dataset generator
def data_generator(dataset, look_back, predict_forward=5, batch_size=32):
    data_len = len(dataset)

    while True:
        X, Y = [], []
        for _ in range(batch_size):
            i = np.random.randint(0, data_len - look_back - predict_forward)
            X.append(dataset[i:i + look_back, 0])
            Y.append(dataset[i + look_back:i + look_back + predict_forward, 0])

        X = np.array(X).reshape(batch_size, look_back, 1)
        Y = np.array(Y)
        yield X, Y

# Create the data generator
train_generator = data_generator(scaled_prices, look_back, predict_forward, batch_size=32)
steps_per_epoch = len(scaled_prices) // 32

# LSTM model definition (reduced size)
def create_model(units=250, optimizer='adam'):  # Reduce units to 128
    model = Sequential()
    model.add(Input(shape=(look_back, 1)))  # Use Input layer instead of specifying input_shape in LSTM
    model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units // 2, return_sequences=False))  # Halve the units for the second LSTM layer
    model.add(Dense(25))
    model.add(Dense(predict_forward, dtype='float32'))  # Mixed precision output as float32
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Use default strategy (e.g., GPU or CPU)
strategy = tf.distribute.get_strategy()

# Early stopping and model checkpointing
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model using the default strategy
with strategy.scope():
    model = create_model(units=250, optimizer='adam')  # Reduced unitsz

    model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        callbacks=[early_stop, checkpoint]
    )

# Save the best model
model.save('best_lstm_model.keras')


Generating train split: 0 examples [00:00, ? examples/s]

Length of minute-level data: 191413
Epoch 1/20
   4/5981 ━━━━━━━━━━━━━━━━━━━━ 4350:03:09 2620s/step - loss: 0.2226